In [1]:
import cv2
import numpy as np
from ipywidgets import *
import matplotlib.pyplot as plt
import math
from math import atan2, cos, sin, sqrt, pi
from sympy import Point, Polygon, Line, Segment

#custom libraries
from TrajectoryPlanner import Node, Planner
from Vision import Vision

In [ ]:
#=== MAIN ===

#Obstacle detection
camera = cv2.VideoCapture(1)
ret, frame = camera.read()
print("Frame shape: " + str(frame.shape))
vision = Vision()
x_center,y_center, goal_detected = vision.get_goal_position(frame)
center, radius, angle, thymio_detected = vision.get_thymio_position(frame,(0,0),0)

vertices = vision.get_obstacle_position(img,radius)[0]

print("Planning")

#Planning
planner = Planner()
planner.setup(vertices)
planner.plot()